In [1]:
%load_ext Cython
import numpy as np
from cython.parallel import prange
import numpy as np
import time
import sys
import os
import torch
import pickle

sys.path.append('C:\\Users\\Nicholas\\Documents\\University @ Glasgow\\Year 5\\cython_test\\retinavision')
import retinavision
from os.path import dirname, join
datadir = os.path.dirname(os.path.abspath("__file__"))
print(datadir)

size = 400
iterations = 100

C:\Users\Nicholas\Documents\University @ Glasgow\Year 5\cython_test\retinavision_cython


In [2]:
def loadPickle(path):
    with open(path, 'rb') as handle:
          return pickle.load(handle, encoding='latin1')

In [3]:
x = join(datadir, "data", "retinas", "ret50k_coeff.pkl")
rf_coeff = loadPickle(x)

In [41]:
original = rf_coeff
no_list = rf_coeff[0]
output_list = []
total_elements = 0
for x in no_list:
    total_elements += x.size
    
for x in no_list:
    one_d = x.ravel()
    output_list.append(one_d)

output_list_2 = [item for sublist in output_list for item in sublist]
arr = np.array(output_list_2)
print(total_elements)
print(no_list[0])
print(arr[0:9])


4706878
[[0.00086751 0.01137962 0.0030938 ]
 [0.03168941 0.41568714 0.11301366]
 [0.02399191 0.31471483 0.08556212]]
[0.00086751 0.01137962 0.0030938  0.03168941 0.41568714 0.11301366
 0.02399191 0.31471483 0.08556212]


In [4]:
%%cython
import numpy as np

cdef cnp.float64_t[:] coeff_memory_view = ravelled

def mandelbrot_cython(mem_view):
    cdef float m = 0
    for i in range(size(mem_view)):
        m += mem_view[i]
        
    return m        


Error compiling Cython file:
------------------------------------------------------------
...
import numpy as np

cdef cnp.float64_t[:] coeff_memory_view = ravelled
    ^
------------------------------------------------------------

C:\Users\Nicholas\.ipython\cython\_cython_magic_862c4b66bb0d92801a1e23919ae2c142.pyx:3:5: 'cnp' is not declared

Error compiling Cython file:
------------------------------------------------------------
...
import numpy as np

cdef cnp.float64_t[:] coeff_memory_view = ravelled
                                         ^
------------------------------------------------------------

C:\Users\Nicholas\.ipython\cython\_cython_magic_862c4b66bb0d92801a1e23919ae2c142.pyx:3:42: undeclared name not builtin: ravelled

Error compiling Cython file:
------------------------------------------------------------
...

cdef cnp.float64_t[:] coeff_memory_view = ravelled

def mandelbrot_cython(mem_view):
    cdef float m = 0
    for i in range(size(mem_view)):
                

In [3]:
s = (size, size)

In [4]:
%%timeit -n1 -r1 m = np.zeros(s, dtype=np.int32)
mandelbrot_cython(m, size, iterations)

5.21 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%cython
import numpy as np

def mandelbrot_cython(int[:,::1] m,
                      int size,
                      int iterations):
    cdef int i, j, n
    cdef complex z, c
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if z.real**2 + z.imag**2 <= 100:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [6]:
%%timeit -n1 -r1 m = np.zeros(s, dtype=np.int32)
mandelbrot_cython(m, size, iterations)

224 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [3]:
from numba import jit

@jit
def numba_mean(x):
    total = 0
    for xi in x:
        total += xi
    return total / len(x)


OSError: Could not load shared object file: llvmlite.dll

In [1]:
 %load_ext Cython

In [2]:
%%cython
def f(n):
    x = 0
    for i in range(n):
        x += i
    return x

##cythonised version
cpdef g(int n):
    cdef long x = 0
    cdef int i
    for i in range(n):
        x += i
    return x

In [3]:
%timeit f(1000000)

37.9 ms ± 3.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%timeit g(1000000)

260 µs ± 3.85 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
